# Nuclei familiari e condizioni di povertà assoluta

## Introduzione
Questa è un'analisi di alcuni [microdati ISTAT sulle spese delle famiglie](https://www.istat.it/it/archivio/180356),
commissionata dall'Università di Bergamo.

In particolare si vogliono estrarre:

* Percentuale di monogenitori in povertà assoluta sul totale dei monogenitori
* Percentuale di famiglie numerose (con 3 e più figli) in povertà assoluta sul totale dello stesso tipo di nucleo.
* Percentuale di famiglie senza figli (solo coppie) in povertà assoluta sullo stesso nucleo
* Percentuale di famiglie unipersonali in povertà assoluta sullo stesso tipo di famiglia.
* La percentuale tra le famiglie monogenitoriali con figli che vivono in una casa di proprietà 
* La percentuale tra le famiglie monogenitoriali con figli che vivono in una casa in affitto
* La percentuale tra le famiglie monogenitoriali con figli che non vanno in vacanza 
* La percentuale tra le famiglie monogenitoriali con figli che andava in vacanza ma non è riuscito ad andare in vacanza


## Preparazione dei dati

I microdati arrivano con oltre 1200 colonne, è possibile semplificarli, considerando solo quelle significative, in modo da rendere rapido l'import e la computazione.

Si considerano solamente i primi 6 set di variabili sui membri del nucleo familiare, dato che nel campione non sono presenti 
record con dati nei set dal 7.mo al 12.mo. 

Sono selezionati i primi 6 set di campi degli individui componenti il nucleo, 
non essendo stati rilevati dati per nuclei con più di 6 elementi nel campione.
ed altre variabili:
 - `Titoccup` - titolo di occupazione della abitazione (1 affitto, 2 proprietà, 2 usufrutto, 4 usufrutto gratuito) 
 - `viaggi_ridqt`, - ha ridotto viaggi e vacanze o non li compra più
 - `viaggi_no_acq` - non ha mai acquistato viaggi e vacanze e continua a non farlo 
 - `povassc` - indicatore di povertà assoluta

Il modo più rapido di tagliare i dati è quello di usare la utility a linea di comando `csvcut`, selezionando esclusivamente 
le colonne cui siamo interessati. 
```
csvcut -c1-80,190,1221,1226,1227,1241  -t HBS_Microdati_2018.txt > semplificati_2018.csv
```




## Import in pandas

In [1]:
import pandas as pd

df = pd.read_csv("./data/HBS_2018_IT_TXT/MICRODATI/semplificati_2018.csv")
n_df = len(df)

Aggiungiamo delle colonne *calcolate*, per semplificare le estrazioni nel seguito:

In [2]:
def count_children(r):
    n = 0
    if r.c_relaz_2 == ' 3':
        n += 1
    if r.c_relaz_3 == ' 3':
        n += 1
    if r.c_relaz_4 == ' 3':
        n += 1
    if r.c_relaz_5 == ' 3':
        n += 1
    if r.c_relaz_6 == ' 3':
        n += 1
    return n
df['n_children'] = df.apply(func=lambda r:count_children(r), axis=1)

In [3]:
def compute_is_mono(r):
    is_mono = False
    if (
        r.c_Ncmp_fatto > 1 and r.n_children > 0 and not (
            r.c_relaz_2 == ' 2' or 
            r.c_relaz_3 == ' 2' or 
            r.c_relaz_4 == ' 2' or 
            r.c_relaz_5 == ' 2' or 
            r.c_relaz_6 == ' 2'
        )
    ):
        is_mono = True
    return is_mono
df['is_mono'] = df.apply(func=lambda r:compute_is_mono(r), axis=1)

Una funzione comune, per calcolare i dati di diversi nuclei:

In [4]:
def compute_data(sub_df, n_tot, article="i", label="Nuclei familiari"):
    n = len(sub_df)
    perc = 100. * n / n_tot
    n_pov = len(sub_df[sub_df.povassc > 0])
    perc_pov_rel = 100. * n_pov / n
    perc_pov_abs = 100. * n_pov / n_tot
    print(f"Numero rilevazioni nel campione: {n_df}.")
    print(f"{label}: {n} ({perc:.2f}% del campione).")
    print(f"  in povertà assoluta: {n_pov} ({perc_pov_rel:.2f}% tra {article} {label.lower()} e {perc_pov_abs:.2f}% del campione)")
    return n, perc, n_pov, perc_pov_rel, perc_pov_abs

## Nuclei monogenitoriali con figli

In [12]:
df_mono = df[df['is_mono']]
n_mono, n_perc_mono, n_mono_pov, perc_mono_pv_rel, perc_mono_pov_abs = compute_data(df_mono, n_df, label="Nuclei monogenitoriali")
n_mono_affitto = len(df_mono[df_mono.Titoccup == 1])
n_mono_proprieta = len(df_mono[df_mono.Titoccup == 2])
n_mono_less_holiday = len(df_mono[df_mono.viaggi_ridqt == '2'])
n_mono_no_holiday = len(df_mono[df_mono.viaggi_no_acq == '7'])
print(f"  in affitto: {n_mono_affitto} ({100.*n_mono_affitto/n_mono:.2f}% tra i nuclei e {100.*n_mono_affitto/n_df:.2f}% del campione)")
print(f"  proprietari: {n_mono_proprieta} ({100.*n_mono_proprieta/n_mono:.2f}% tra i nuclei e {100.*n_mono_proprieta/n_df:.2f}% del campione)")
print(f"  vanno meno in vacanza: {n_mono_less_holiday} ({100.*n_mono_less_holiday/n_mono:.2f}% tra i nuclei e {100.*n_mono_less_holiday/n_df:.2f}% del campione)")
print(f"  non vanno in vacanza: {n_mono_no_holiday} ({100.*n_mono_no_holiday/n_mono:.2f}% tra i nuclei e {100.*n_mono_no_holiday/n_df:.2f}% del campione)")

Numero rilevazioni nel campione: 18342.
Nuclei monogenitoriali: 1617 (8.82% del campione).
  in povertà assoluta: 182 (11.26% tra i nuclei monogenitoriali e 0.99% del campione)
  in affitto: 378 (23.38% tra i nuclei e 2.06% del campione)
  proprietari: 1079 (66.73% tra i nuclei e 5.88% del campione)
  vanno meno in vacanza: 319 (19.73% tra i nuclei e 1.74% del campione)
  non vanno in vacanza: 753 (46.57% tra i nuclei e 4.11% del campione)


## Famiglie numerose (>3 figli)

In [55]:
df_num = df[df.n_children >= 3]
compute_data(df_num, n_df, label="Famiglie numerose", article="le")

Numero rilevazioni nel campione: 18342.
Famiglie numerose: 671 (3.66% del campione).
  di cui in povertà assoluta: 117 (17.44% tra le famiglie numerose e 0.64% del campione)


## Coppie senza figli

In [56]:
df_nc = df[(df.c_Ncmp_fatto > 1) & (df.n_children == 0)]
compute_data(df_nc, n_df, label="Coppie senza figli", article="le")

Numero rilevazioni nel campione: 18342.
Coppie senza figli: 5087 (27.73% del campione).
  di cui in povertà assoluta: 226 (4.44% tra le coppie senza figli e 1.23% del campione)


## Nuclei unipersonali

In [57]:
df_uni = df[df.c_Ncmp_fatto == 1]
compute_data(df_uni, n_df, label="Nuclei unipersonali", article="i")

Numero rilevazioni nel campione: 18342.
Nuclei unipersonali: 5733 (31.26% del campione).
  di cui in povertà assoluta: 327 (5.70% tra i nuclei unipersonali e 1.78% del campione)
